In [37]:
import sys
sys.path.insert(0, "..")
from pathlib import Path

from src.data import make_dataset
from src.models import train_model
from src.models import metrics

import torch
from torch import nn
import torch.nn as nn
import torch.optim as optim

import gin
sys.path.insert(0, "../..")

In [38]:
datadir = Path("../../data/raw/")
train_dataloader, test_dataloader = make_dataset.get_MNIST(datadir, batch_size=64) 
len(train_dataloader), len(test_dataloader)

(938, 157)

In [39]:
#gin.parse_config_file("model.gin")

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Bij deze opdracht werken we met verschillende neurale netwerken. met 2, 3 of 4 Convolutional layers.
Daarnaast met 2, 3 of 4 lineaire lagen.

In [54]:
# Define model
# Dit model is uitgangspunt & komt uit 07_exercise (les 1)
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

model = CNN().to(device)

In [55]:
print(model)

CNN(
  (convolutions): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=10, bias=True)
  )
)


In [51]:
# Define model 2: 2 Conv layers
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            #nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2),
        )
        
        #self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, padding=2)
        #self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

model2 = CNN().to(device)


In [52]:
print(model2)

CNN(
  (convolutions): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=10, bias=True)
  )
)


In [48]:
train_model.count_parameters(model2)

20234

In [47]:
x, y = next(iter(train_dataloader))
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [ ]:
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [53]:
# dit komt uit 07_exercises.ipynb. Het Gin-bestand matched hier niet mee.
# Dit Gin-bestand komt origineel uit: uit les 1. Dus beiden zouden samen moeten gaan.

import torch.optim as optim
#gin.parse_config_file("model.gin")

units = [512, 256, 128]
learning_rate = [0.01, 0.001, 0.0001]

for unit in units:
    for lr in learning_rate:
        #gin.bind_parameter("NeuralNetwork.units2", unit)
        #gin.bind_parameter("trainloop.learning_rate", lr)
    
        #model = imagemodels.NeuralNetwork()
        model = model2

        model =  train_model.trainloop(
            epochs=3,
            model=model,
            optimizer=optimizer,
            loss_fn=loss_fn,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            log_dir="../../models/test/",
            train_steps=len(train_dataloader),
            eval_steps=150,    
        )

2022-12-13 09:35:19.790 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/test/20221213-0935


mat1 and mat2 shapes cannot be multiplied (64x1152 and 128x64)
Error occurs, No graph saved


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x1152 and 128x64)
  In call to configurable 'trainloop' (<function trainloop at 0x7f7354ec7670>)

In [ ]:
model = train_model.trainloop(
    epochs=3,
    model=model,
    optimizer=optimizer,
    learning_rate=1e-3,
    loss_fn=loss_fn,
    metrics=[accuracy],
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    log_dir="../../models/test/",
    train_steps=len(train_dataloader),
    eval_steps=len(test_dataloader),
)